In [6]:

from binance.client import Client
import pandas as pd

my_api = "OkNTS53iVnQwQ4SvVn9rScwpqGbuX37uzAboebgssjVC53ZPBBOIycYE7tWcgKoA"
my_secret = "lIStreQbr4ob7I6x5A7DpOJHIRmb5shNDIve79H56A7kAfhcDoe1qLapUnHF4wt1"

client = Client(api_key=my_api, api_secret=my_secret, tld="com", testnet=True)

In [5]:
account = client.get_account()
account

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1654620276062,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [7]:
df = pd.DataFrame(account["balances"])
df

,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [8]:
df.free = pd.to_numeric(df.free, errors="coerce")
df.locked = pd.to_numeric(df.locked, errors="coerce")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   asset   8 non-null      object 
 1   free    8 non-null      float64
 2   locked  8 non-null      object 
dtypes: float64(1), object(2)
memory usage: 320.0+ bytes


In [10]:
client.get_asset_balance(asset="BTC")

{'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}

In [11]:
client.get_asset_balance(asset="ETH")

{'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}

In [14]:
client.get_asset_balance(asset="USDT")

{'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'}

In [15]:
client.get_symbol_ticker(symbol="BTCUSDT") # 하나의 심볼에 대한 현재 가격

{'symbol': 'BTCUSDT', 'price': '29919.07000000'}

In [17]:
last24 = client.get_all_tickers()
last24# 모든 페어의 현재 가격을 다 가져온다.

[{'symbol': 'BNBBUSD', 'price': '283.80000000'},
 {'symbol': 'BTCBUSD', 'price': '29904.77000000'},
 {'symbol': 'ETHBUSD', 'price': '1781.72000000'},
 {'symbol': 'LTCBUSD', 'price': '62.20000000'},
 {'symbol': 'TRXBUSD', 'price': '0.08123000'},
 {'symbol': 'XRPBUSD', 'price': '0.39470000'},
 {'symbol': 'BNBUSDT', 'price': '284.00000000'},
 {'symbol': 'BTCUSDT', 'price': '29925.60000000'},
 {'symbol': 'ETHUSDT', 'price': '1783.40000000'},
 {'symbol': 'LTCUSDT', 'price': '62.30000000'},
 {'symbol': 'TRXUSDT', 'price': '0.08131000'},
 {'symbol': 'XRPUSDT', 'price': '0.39500000'},
 {'symbol': 'BNBBTC', 'price': '0.00949000'},
 {'symbol': 'ETHBTC', 'price': '0.05958500'},
 {'symbol': 'LTCBTC', 'price': '0.00208300'},
 {'symbol': 'TRXBTC', 'price': '0.00000272'},
 {'symbol': 'XRPBTC', 'price': '0.00001321'},
 {'symbol': 'LTCBNB', 'price': '0.22030000'},
 {'symbol': 'TRXBNB', 'price': '0.00028640'},
 {'symbol': 'XRPBNB', 'price': '0.00139200'}]

In [19]:
timestamp = client._get_earliest_valid_timestamp(symbol="BTCUSDT", interval = "1d")
timestamp#가장 처음 타임 스탬프를 가져올 수 있다 + 유닉스 기반의 시간으로

1651622400000

In [20]:
pd.to_datetime(timestamp, unit="ms")##유닉스 형 시간을 일반 시간으로 변경한다.

Timestamp('2022-05-04 00:00:00')

In [27]:
def get_history(symbol, interval, start, end = None):
    bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                        start_str = start, end_str = end, limit = 1000)
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                  "Clos Time", "Quote Asset Volume", "Number of Trades",
                  "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace = True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors = "coerce")
    
    return df
    

In [28]:
df = get_history(symbol="BTCUSDT", interval="1d", start=timestamp)
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-05-04,38127.16,67952.01,9000.00,39695.80,3289.478889
2022-05-05,39690.01,48256.09,12000.00,36551.04,3747.859117
2022-05-06,36552.97,90000.00,9500.00,35844.54,5133.878780
2022-05-07,35851.70,179685.00,2000.00,35472.39,2648.748370
2022-05-08,35472.40,172120.00,6988.00,34035.00,5928.739345
2022-05-09,34036.87,44600.00,6392.00,30092.43,7687.567239
2022-05-10,30089.10,34490.00,10000.00,31017.10,8559.000857
2022-05-11,31016.84,158649.00,5937.00,29098.29,8773.143745
2022-05-12,29098.29,30194.46,10000.00,29025.46,9821.112945


In [43]:
df = get_history(symbol="BTCUSDT", interval="1h", start = "2021-09-15")
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-05-04 06:00:00,38127.16,38421.04,15632.10,38408.81,69.302088
2022-05-04 07:00:00,38409.20,48200.00,9000.00,38702.86,186.261019
2022-05-04 08:00:00,38701.31,39025.24,28200.00,38969.37,200.229146
2022-05-04 09:00:00,38969.37,48200.00,28200.00,38877.68,173.722128
2022-05-04 10:00:00,38884.13,39211.83,28200.00,38989.83,193.351971
...,...,...,...,...,...
2022-06-07 13:00:00,29504.14,29701.18,29430.02,29564.47,228.639495
2022-06-07 14:00:00,29564.48,30022.72,29514.04,29904.80,284.686331
2022-06-07 15:00:00,29904.43,30056.99,29778.24,29891.41,273.298635


In [30]:
from binance import ThreadedWebsocketManager



In [31]:
def stream_data(msg):
    time = pd.to_datetime(msg["E"], unit="ms")
    price = msg["c"]
    
    print("Time = {}, price : {}".format(time, price))

In [1]:
twm = ThreadedWebsocketManager()
twm.start()

NameError: name 'ThreadedWebsocketManager' is not defined

In [ ]:
twm.start_symbol_miniticker_socket(callback=stream_data, symbol="BTCUSDT")

In [35]:
twm.stop()

매입 주문 넣기

In [45]:
from binance.client import Client
import pandas as pd

In [46]:
client = Client(api_key=my_api, api_secret=my_secret, tld="com", testnet=True)


In [47]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1654620276062,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [48]:
order = client.create_order(symbol="BTCUSDT", side="BUY", type="MARKET", quantity=0.1)

In [49]:
order

{'symbol': 'BTCUSDT',
 'orderId': 14016986,
 'orderListId': -1,
 'clientOrderId': 'XIfEXSyGK2X41X9HahoMT7',
 'transactTime': 1654624777786,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '2994.49914843',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'fills': [{'price': '29944.99000000',
   'qty': '0.08515700',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 5486111},
  {'price': '29945.00000000',
   'qty': '0.01484300',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 5486112}]}

In [51]:
df = pd.DataFrame(order["fills"])
df

,price,qty,commission,commissionAsset,tradeId
0,29944.99000000,0.08515700,0.00000000,BTC,5486111
1,29945.00000000,0.01484300,0.00000000,BTC,5486112


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   price            2 non-null      object
 1   qty              2 non-null      object
 2   commission       2 non-null      object
 3   commissionAsset  2 non-null      object
 4   tradeId          2 non-null      int64 
dtypes: int64(1), object(4)
memory usage: 208.0+ bytes


In [53]:
num_columns = ["price", "qty", "commission"]

In [54]:
for column in num_columns:
    df[column] = pd.to_numeric(df[column], errors="coerce")

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            2 non-null      float64
 1   qty              2 non-null      float64
 2   commission       2 non-null      float64
 3   commissionAsset  2 non-null      object 
 4   tradeId          2 non-null      int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 208.0+ bytes


In [56]:
df.qty.sum()

0.09999999999999999

In [57]:
df.price.mul(df.qty).sum()

2994.49914843

In [58]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1654624777786,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.10000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '7005.50085157', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [60]:
# buy BTC for 1000 USDT

order = client.create_order(symbol="BTCUSDT", side="BUY", type="MARKET", quoteOrderQty=1000)
order

{'symbol': 'BTCUSDT',
 'orderId': 14019513,
 'orderListId': -1,
 'clientOrderId': 'bVvWko69zx7JcHy1cFsZ7G',
 'transactTime': 1654625349325,
 'price': '0.00000000',
 'origQty': '0.03336300',
 'executedQty': '0.03336300',
 'cummulativeQuoteQty': '999.98819292',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'fills': [{'price': '29972.84000000',
   'qty': '0.02794800',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 5487040},
  {'price': '29973.64000000',
   'qty': '0.00541500',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 5487041}]}

In [61]:
aver_price = float(order["cummulativeQuoteQty"]) / float(order["origQty"])
aver_price

29972.969844438452

In [62]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1654625349325,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.13336300', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '6005.51265865', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [65]:
# 매도 주문 넣기
order = client.create_order(symbol="BTCUSDT", side="SELL", type="MARKET", quantity = 0.11811)
order

{'symbol': 'BTCUSDT',
 'orderId': 14020864,
 'orderListId': -1,
 'clientOrderId': 'yTkbgVNMwk4qBKLCY6mICK',
 'transactTime': 1654625648135,
 'price': '0.00000000',
 'origQty': '0.11811000',
 'executedQty': '0.11811000',
 'cummulativeQuoteQty': '3559.80276635',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'fills': [{'price': '30140.71000000',
   'qty': '0.04404400',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 5487741},
  {'price': '30139.14000000',
   'qty': '0.05325300',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 5487742},
  {'price': '30139.13000000',
   'qty': '0.02081300',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 5487743}]}

In [66]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1654625648135,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.01525300', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9565.31542500', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [67]:
client.get_symbol_ticker(symbol="BTCUSDT")

{'symbol': 'BTCUSDT', 'price': '30144.99000000'}

In [90]:
#지정가 매도 주문 넣기
order1 = client.create_order(symbol="BTCUSDT", side="BUY", type="LIMIT", quantity=0.1, timeInForce="GTC", price=50000)
#GTC = "주문이 체결되거나 취소될때까지 유지하겠다는 의미이다.
order1

BinanceAPIException: APIError(code=-2010): Account has insufficient balance for requested action.

In [88]:
order2 = client.create_order(symbol="BTCUSDT", side="SELL", type="LIMIT", quantity=0.1, timeInForce="GTC", price=60000)
order2

{'symbol': 'BTCUSDT',
 'orderId': 14023994,
 'orderListId': -1,
 'clientOrderId': 'KR14uDzNGdkQkekhqB9e0r',
 'transactTime': 1654626351517,
 'price': '60000.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL',
 'fills': []}

In [80]:
open_orders = client.get_open_orders(symbol="BTCUSDT")
open_orders

[]

In [81]:
pd.DataFrame(open_orders)##미체결 주문을 가져올 수 있다.

""


In [74]:
order_id_sell = order2["orderId"]
order_id_sell

14022385

In [93]:
all_orders = client.get_all_orders(symbol="BTCUSDT")
all_orders

[{'symbol': 'BTCUSDT',
  'orderId': 14016986,
  'orderListId': -1,
  'clientOrderId': 'XIfEXSyGK2X41X9HahoMT7',
  'price': '0.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.10000000',
  'cummulativeQuoteQty': '2994.49914843',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1654624777786,
  'updateTime': 1654624777786,
  'isWorking': True,
  'origQuoteOrderQty': '0.00000000'},
 {'symbol': 'BTCUSDT',
  'orderId': 14019513,
  'orderListId': -1,
  'clientOrderId': 'bVvWko69zx7JcHy1cFsZ7G',
  'price': '0.00000000',
  'origQty': '0.03336300',
  'executedQty': '0.03336300',
  'cummulativeQuoteQty': '999.98819292',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1654625349325,
  'updateTime': 1654625349325,
  'isWorking': True,
  'origQuoteOrderQty': '1000.00000000'},
 {'symbol'

In [95]:
pd.DataFrame(all_orders)

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,origQuoteOrderQty
0,BTCUSDT,14016986,-1,XIfEXSyGK2X41X9HahoMT7,0.00000000,0.10000000,0.10000000,2994.49914843,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1654624777786,1654624777786,True,0.00000000
1,BTCUSDT,14019513,-1,bVvWko69zx7JcHy1cFsZ7G,0.00000000,0.03336300,0.03336300,999.98819292,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1654625349325,1654625349325,True,1000.00000000
2,BTCUSDT,14020864,-1,yTkbgVNMwk4qBKLCY6mICK,0.00000000,0.11811000,0.11811000,3559.80276635,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1654625648135,1654625648135,True,0.00000000
3,BTCUSDT,14021745,-1,UFGIaItqlsxTIj56EDq9iO,50000.00000000,0.10000000,0.10000000,3013.39321614,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,1654625849232,1654625849232,True,0.00000000
4,BTCUSDT,14022385,-1,13Uh7gMldghHlf7BBi8TLU,60000.00000000,0.10000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,SELL,0.00000000,0.00000000,1654625992629,1654626244476,True,0.00000000
5,BTCUSDT,14023900,-1,0T3vrx9xQle23FMDsNhhV0,50000.00000000,0.10000000,0.10000000,3014.89800000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,1654626330106,1654626330106,True,0.00000000
6,BTCUSDT,14023994,-1,KR14uDzNGdkQkekhqB9e0r,60000.00000000,0.10000000,0.00000000,0.00000000,NEW,GTC,LIMIT,SELL,0.00000000,0.00000000,1654626351517,1654626351517,True,0.00000000


In [96]:
trades = client.get_my_trades(symbol="BTCUSDT")
trades

[{'symbol': 'BTCUSDT',
  'id': 5486111,
  'orderId': 14016986,
  'orderListId': -1,
  'price': '29944.99000000',
  'qty': '0.08515700',
  'quoteQty': '2550.02551343',
  'commission': '0.00000000',
  'commissionAsset': 'BTC',
  'time': 1654624777786,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 5486112,
  'orderId': 14016986,
  'orderListId': -1,
  'price': '29945.00000000',
  'qty': '0.01484300',
  'quoteQty': '444.47363500',
  'commission': '0.00000000',
  'commissionAsset': 'BTC',
  'time': 1654624777786,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 5487040,
  'orderId': 14019513,
  'orderListId': -1,
  'price': '29972.84000000',
  'qty': '0.02794800',
  'quoteQty': '837.68093232',
  'commission': '0.00000000',
  'commissionAsset': 'BTC',
  'time': 1654625349325,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 5487041,
  'orderId': 14019513,


In [97]:
pd.DataFrame(trades)

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch
0,BTCUSDT,5486111,14016986,-1,29944.99000000,0.08515700,2550.02551343,0.00000000,BTC,1654624777786,True,False,True
1,BTCUSDT,5486112,14016986,-1,29945.00000000,0.01484300,444.47363500,0.00000000,BTC,1654624777786,True,False,True
2,BTCUSDT,5487040,14019513,-1,29972.84000000,0.02794800,837.68093232,0.00000000,BTC,1654625349325,True,False,True
3,BTCUSDT,5487041,14019513,-1,29973.64000000,0.00541500,162.30726060,0.00000000,BTC,1654625349325,True,False,True
4,BTCUSDT,5487741,14020864,-1,30140.71000000,0.04404400,1327.51743124,0.00000000,USDT,1654625648135,False,False,True
5,BTCUSDT,5487742,14020864,-1,30139.14000000,0.05325300,1604.99962242,0.00000000,USDT,1654625648135,False,False,True
6,BTCUSDT,5487743,14020864,-1,30139.13000000,0.02081300,627.28571269,0.00000000,USDT,1654625648135,False,False,True
7,BTCUSDT,5488156,14021745,-1,30133.93000000,0.07838600,2362.07823698,0.00000000,BTC,1654625849232,True,False,True
8,BTCUSDT,5488157,14021745,-1,30133.94000000,0.02161400,651.31497916,0.00000000,BTC,1654625849232,True,False,True
9,BTCUSDT,5489217,14023900,-1,30148.98000000,0.08557600,2580.02911248,0.00000000,BTC,1654626330106,True,False,True
